In [1]:
%cd ..

/mnt/SSD_Data/active_projects/transformer_to_lstm


In [2]:
import dataclasses
from pathlib import Path
import warnings

import nlp
import torch
import numpy as np
import torch.nn.functional as F
from transformers import BertForSequenceClassification
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split

from pytorch_helper_bot import (
    BaseBot, MovingAverageStatsTrackerCallback,  CheckpointCallback,
    LearningRateSchedulerCallback, MultiStageScheduler, Top1Accuracy,
    LinearLR, Callback
)

from nobita.models import get_sequence_model


try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False
# APEX_AVAILABLE = False

In [3]:
CACHE_DIR = Path("cache/")
CACHE_DIR.mkdir(exist_ok=True)

Reference:

    * https://github.com/huggingface/nlp/blob/master/notebooks/Overview.ipynb

In [4]:
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, entries_dict):
        super().__init__()
        self.entries_dict = entries_dict
    
    def __len__(self):
        return len(self.entries_dict["label"])
    
    def __getitem__(self, idx):
        return (
            self.entries_dict["input_ids"][idx],
            self.entries_dict["attention_mask"][idx].sum(), #input_lengths
            {
                "label": self.entries_dict["label"][idx], 
                "logits": self.entries_dict["logits"][idx]
            }
        )

In [5]:
train_dict, valid_dict, test_dict = torch.load(str(CACHE_DIR / "distill-dicts.jbl"))

In [6]:
# Instantiate a PyTorch Dataloader around our dataset
train_loader = torch.utils.data.DataLoader(SST2Dataset(train_dict), batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(SST2Dataset(valid_dict), batch_size=64, drop_last=False)
test_loader = torch.utils.data.DataLoader(SST2Dataset(test_dict), batch_size=64, drop_last=False)

In [7]:
ALPHA = 1.
DISTILL_OBJECTIVE = torch.nn.MSELoss()

def distill_loss(logits, targets):
    distill_part = DISTILL_OBJECTIVE(
        logits.reshape(-1), targets["logits"].reshape(-1)
    ) / 2
    classification_part = F.cross_entropy(
        logits, targets["label"]
    )
    return ALPHA * classification_part + (1-ALPHA) * distill_part

In [8]:
bert_model = BertForSequenceClassification.from_pretrained(str(CACHE_DIR / "sst2_bert_uncased")).cpu()

In [9]:
bert_model.bert.embeddings.word_embeddings.weight.shape

torch.Size([30522, 768])

In [10]:
# Note: apex does not support weight dropping
model = get_sequence_model(
    voc_size=bert_model.bert.embeddings.word_embeddings.weight.shape[0],
    emb_size=bert_model.bert.embeddings.word_embeddings.weight.shape[1],
    pad_idx = 0,
    dropoute = 0,
    rnn_hid = 768,
    rnn_layers = 3,
    bidir = True,
    dropouth = 0.25,
    dropouti = 0.25,
    wdrop = 0,
    unit_type = "lstm",
    fcn_layers = [512, 2],
    fcn_dropouts = [0.25, 0.25],
    use_attention = True
)

In [11]:
model

SequenceModel(
  (embeddings): BasicEmbeddings(
    (encoder): Embedding(30522, 768, padding_idx=0)
  )
  (encoder): RNNStack(
    (rnns): ModuleList(
      (0): LSTM(768, 384, bidirectional=True)
      (1): LSTM(768, 384, bidirectional=True)
      (2): LSTM(768, 384, bidirectional=True)
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
    )
  )
  (fcn): AttentionFCN(
    (attention): Attention(768, return attention=False)
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=768, out_features=512, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): LinearBlock(
        (lin): Linear(in_features=512, out_features=2, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, 

In [12]:
# Copy the embedding weights to the LSTM model
try:
    model.embeddings.encoder.emb.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data
except:
    model.embeddings.encoder.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data

In [13]:
# Freeze the embedding layer
for param in model.embeddings.encoder.parameters():
    param.requires_grad = False

In [14]:
model = model.cuda()

In [15]:
# Use only leaf tensors
parameters = [x for x in model.parameters() if x.is_leaf and x.requires_grad]

In [16]:
del bert_model

In [17]:
optimizer = torch.optim.Adam(parameters, lr=1e-3, betas=(0.8, 0.99))
# optimizer = torch.optim.RMSprop(parameters, lr=0.01)

In [18]:
if APEX_AVAILABLE:
    model, optimizer = amp.initialize(
        model, optimizer, opt_level="O1"
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [19]:
class TransposeCallback(Callback):
    def on_batch_inputs(self, bot, input_tensors, targets):
        input_tensors = [input_tensors[0].transpose(1, 0), input_tensors[1]]
        return input_tensors, targets

In [20]:
class DistillTop1Accuracy(Top1Accuracy):
    def __call__(self, truth, pred):
        truth = truth["label"]
        return super().__call__(truth, pred)

In [21]:
total_steps = len(train_loader) * 10

checkpoints = CheckpointCallback(
    keep_n_checkpoints=1,
    checkpoint_dir=CACHE_DIR / "distill_model_cache/",
    monitor_metric="loss"
)
lr_durations = [
    int(total_steps*0.2),
    int(np.ceil(total_steps*0.8))
]
break_points = [0] + list(np.cumsum(lr_durations))[:-1]
callbacks = [
    MovingAverageStatsTrackerCallback(
        avg_window=len(train_loader) // 8,
        log_interval=len(train_loader) // 10
    ),
    LearningRateSchedulerCallback(
        MultiStageScheduler(
            [
                LinearLR(optimizer, 0.01, lr_durations[0]),
                CosineAnnealingLR(optimizer, lr_durations[1])
            ],
            start_at_epochs=break_points
        )
    ),
    checkpoints,
    TransposeCallback()
]
    
bot = BaseBot(
    log_dir = CACHE_DIR / "distill_logs",
    model=model, 
    train_loader=train_loader,
    valid_loader=valid_loader, 
    clip_grad=10.,
    optimizer=optimizer, echo=True,
    criterion=distill_loss,
    callbacks=callbacks,
    pbar=False, use_tensorboard=False,
    use_amp=APEX_AVAILABLE,
    metrics=(DistillTop1Accuracy(),)
)

[INFO][06/15/2020 21:12:21] SEED: 9293
[INFO][06/15/2020 21:12:21] # of parameters: 34,474,242
[INFO][06/15/2020 21:12:21] # of trainable parameters: 11,033,346


In [22]:
print(total_steps)

bot.train(
    total_steps=total_steps,
    checkpoint_interval=len(train_loader) // 2
)
bot.load_model(checkpoints.best_performers[0][1])
checkpoints.remove_checkpoints(keep=0)

[INFO][06/15/2020 21:12:21] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.8, 0.99)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0
)
[INFO][06/15/2020 21:12:21] Batches per epoch: 1053
[INFO][06/15/2020 21:12:21] ====================Epoch 1====================


10530


[INFO][06/15/2020 21:12:24] Step   105 | loss 1.17822204 | lr: 5.94e-05 | 0.029s per step
[INFO][06/15/2020 21:12:27] Step   210 | loss 0.98571526 | lr: 1.09e-04 | 0.028s per step
[INFO][06/15/2020 21:12:29] Step   315 | loss 0.84693433 | lr: 1.58e-04 | 0.027s per step
[INFO][06/15/2020 21:12:32] Step   420 | loss 0.71049504 | lr: 2.08e-04 | 0.028s per step
[INFO][06/15/2020 21:12:35] Step   525 | loss 0.62278121 | lr: 2.57e-04 | 0.027s per step
[INFO][06/15/2020 21:12:35] Metrics at step 526:
[INFO][06/15/2020 21:12:35] loss: 0.73929630
[INFO][06/15/2020 21:12:35] accuracy: 56.19%
[INFO][06/15/2020 21:12:38] Step   630 | loss 0.57960875 | lr: 3.06e-04 | 0.030s per step
[INFO][06/15/2020 21:12:41] Step   735 | loss 0.53256572 | lr: 3.56e-04 | 0.027s per step
[INFO][06/15/2020 21:12:44] Step   840 | loss 0.49936575 | lr: 4.05e-04 | 0.027s per step
[INFO][06/15/2020 21:12:47] Step   945 | loss 0.48621889 | lr: 4.54e-04 | 0.027s per step
[INFO][06/15/2020 21:12:50] Step  1050 | loss 0.454

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


[INFO][06/15/2020 21:12:56] Step  1260 | loss 0.50986822 | lr: 6.03e-04 | 0.027s per step
[INFO][06/15/2020 21:12:59] Step  1365 | loss 0.49882624 | lr: 6.52e-04 | 0.027s per step
[INFO][06/15/2020 21:13:01] Step  1470 | loss 0.43877402 | lr: 7.01e-04 | 0.027s per step
[INFO][06/15/2020 21:13:04] Step  1575 | loss 0.42520519 | lr: 7.51e-04 | 0.027s per step
[INFO][06/15/2020 21:13:05] Metrics at step 1578:
[INFO][06/15/2020 21:13:05] loss: 0.47600311
[INFO][06/15/2020 21:13:05] accuracy: 81.42%
[INFO][06/15/2020 21:13:08] Step  1680 | loss 0.42487729 | lr: 8.00e-04 | 0.031s per step
[INFO][06/15/2020 21:13:10] Step  1785 | loss 0.42041942 | lr: 8.50e-04 | 0.027s per step
[INFO][06/15/2020 21:13:13] Step  1890 | loss 0.39767750 | lr: 8.99e-04 | 0.027s per step
[INFO][06/15/2020 21:13:16] Step  1995 | loss 0.38604234 | lr: 9.48e-04 | 0.027s per step
[INFO][06/15/2020 21:13:19] Step  2100 | loss 0.37510548 | lr: 9.98e-04 | 0.027s per step
[INFO][06/15/2020 21:13:19] Metrics at step 2104:


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


[INFO][06/15/2020 21:16:43] Step  9240 | loss 0.14902027 | lr: 5.69e-05 | 0.029s per step
[INFO][06/15/2020 21:16:47] Step  9345 | loss 0.15236143 | lr: 4.82e-05 | 0.030s per step
[INFO][06/15/2020 21:16:50] Step  9450 | loss 0.15284705 | lr: 4.01e-05 | 0.030s per step
[INFO][06/15/2020 21:16:50] Metrics at step 9468:
[INFO][06/15/2020 21:16:50] loss: 0.34211946
[INFO][06/15/2020 21:16:50] accuracy: 86.93%
[INFO][06/15/2020 21:16:51] ====================Epoch 10====================
[INFO][06/15/2020 21:16:53] Step  9555 | loss 0.16034573 | lr: 3.28e-05 | 0.030s per step
[INFO][06/15/2020 21:16:56] Step  9660 | loss 0.14514064 | lr: 2.62e-05 | 0.030s per step
[INFO][06/15/2020 21:16:59] Step  9765 | loss 0.14388169 | lr: 2.03e-05 | 0.029s per step
[INFO][06/15/2020 21:17:02] Step  9870 | loss 0.14752400 | lr: 1.51e-05 | 0.028s per step
[INFO][06/15/2020 21:17:05] Step  9975 | loss 0.14684270 | lr: 1.07e-05 | 0.028s per step
[INFO][06/15/2020 21:17:05] Metrics at step 9994:
[INFO][06/15/

In [23]:
# TARGET_DIR = CACHE_DIR / "sst2_bert_uncased"
# TARGET_DIR.mkdir(exist_ok=True)
# bot.model.save_pretrained(TARGET_DIR)

In [24]:
bot.eval(valid_loader)

{'loss': (0.29697493019453974, '0.29697493'),
 'accuracy': (-0.8807339449541285, '88.07%')}

In [25]:
bot.eval(test_loader)

{'loss': (0.3411313125846583, '0.34113131'),
 'accuracy': (-0.8577981651376146, '85.78%')}

In [26]:
# tokenizer.pad_token_id